<a href="https://colab.research.google.com/github/mlabonne/how-to-data-science/blob/main/Visualizing_GPT_2's_Loss_Landscape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualizing GPT-2's Loss Landscape

❤️ Created by [@maximelabonne](https://twitter.com/maximelabonne).

Simple perturbation-based calculation of the negative log-likelihood loss in two directions, given "I have a dream" as input.

Reference: [Visualizing the Loss Landscape of Neural Nets](https://arxiv.org/abs/1712.09913), by Li et al. (2018)

In [1]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.3 MB/s eta 0:00:00


In [11]:
%%time

import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import plotly.graph_objects as go
from tqdm import tqdm
import imageio
import os

# Load pre-trained model
model_name = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Set model to evaluation mode
model.eval()

# Define our input
input_text = "I have a dream"
inputs = tokenizer.encode_plus(input_text, return_tensors="pt")

# Compute the original loss
outputs = model(**inputs, labels=inputs["input_ids"])
original_loss = outputs.loss.item()

# Define two random directions
direction1 = [torch.randn_like(p) for p in model.parameters()]
direction2 = [torch.randn_like(p) for p in model.parameters()]

# Normalize vectors
for p, d1, d2 in zip(model.parameters(), direction1, direction2):
    norm_p = torch.linalg.norm(p.flatten())
    d1.div_(torch.linalg.norm(d1.flatten())).mul_(norm_p)
    d2.div_(torch.linalg.norm(d2.flatten())).mul_(norm_p)

# Define the range to explore
x = np.linspace(-1, 1, 20)
y = np.linspace(-1, 1, 20)
X, Y = np.meshgrid(x, y)

# Prepare to collect the losses
Z = np.zeros_like(X)

# Compute loss for each direction
for i in tqdm(range(x.size), desc="x progress"):
    for j in tqdm(range(y.size), desc="y progress", leave=False):
        # Perturb the model parameters
        for p, d1, d2 in zip(model.parameters(), direction1, direction2):
            p.data.add_(x[i]*d1 + y[j]*d2)
        
        # Compute the loss
        outputs = model(**inputs, labels=inputs['input_ids'])
        Z[i, j] = outputs.loss.item()
        
        # Revert the model parameters
        for p, d1, d2 in zip(model.parameters(), direction1, direction2):
            p.data.sub_(x[i]*d1 + y[j]*d2)


100%|██████████| 20/20 [12:42<00:00, 38.11s/it]


In [32]:
# Create 3D plot
fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y, 
                                 showscale=False,)])
fig.update_layout(
    title="GPT-2's Loss Landscape",
    autosize=True,
    width=1000,
    height=600,
    # scene=dict(
    #     xaxis=dict(visible=False),
    #     yaxis=dict(visible=False),
    #     zaxis=dict(visible=False),
    # )
)
fig.show()